### Capstone Project - Assignment week 3c: Neighborhood Segmentation and Clustering in Toronto

This weeks assignment is about explore and cluster the neighborhoods in Toronto:  
1) Get Data from Wiki importing required libraries to get data from wiki  
2) Clean the data and make a pandas dataframe  
3) Use the Geocoder package to find coordinates (lat/long)  
4) Explore and cluster the neighborhoods in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

1. <a href="#section1">Download, Explore and Clean Dataset</a>
2. <a href="#section2">Get the coordinates of the neighborhoods</a>
3. <a href="#section3">Explorer the neighborhoods in Toronto</a>
4. <a href="#section4">Cluster the neighborhoods in Toronto</a>


</font>
</div>

<a id="section1"></a>
# 1. Download, Explore and Clean Dataset

#### Import the modules

In [1]:
# Import modules
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml
import csv
from collections import Counter

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#### Get the data from the web page using request

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

#### The website data can be view in a pretty format by using beautifulsoup's prettify() method. 

In [3]:
soup = BeautifulSoup(page.content, 'lxml')

#### The first task is to find class ‘wikitable sortable’ in the script. So, first we will extract the data in table tag using find method of bs4 object.

In [4]:
my_table = soup.find('table', class_='wikitable')

In [5]:
links = my_table.findAll('td')
links

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights

In [6]:
tag = soup.a

In [7]:
tag.attrs

{'id': 'top'}

In [8]:
# Let’s get all of the HTML tags from inside the body of the page.
tags = [tag.name for tag in soup.body.find_all(True)]
len(tags)

1895

In [9]:
# Count up how many of each tag type are present.
tag_counts = Counter(tags)
len(tag_counts)

27

In [10]:
tag_counts.most_common(5)

[('td', 901), ('a', 468), ('tr', 295), ('li', 62), ('div', 43)]

#### Declaring Empty Dataframe with three columns

In [11]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
column_names

['PostalCode', 'Borough', 'Neighborhood']

#### Reading Data from tags transform the data into a pandas dataframe. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [12]:
tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    
    #print(len(columns), columns)
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Only process the cells that have an assigned borough. Ignored the cells with a borough that is not assigned.

In [13]:
neighborhoods.shape

(289, 3)

In [14]:
neighborhoods['Borough'].value_counts()['Not assigned']

77

In [15]:
df_neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("[^Not assigned]")==True]
df_neighborhoods.shape

(212, 3)

In [16]:
# Check: 289 (total lenght of list) - 77 (rows with not assigned borough, 
# ignored) = 212, so far so good

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [63]:
df_neighborhoods['Neighborhood'].replace(to_replace="Not assigned", value=df_neighborhoods['Borough'], inplace=True)

In [18]:
df_neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [19]:
df_neighborhoods.shape

(212, 3)

In [20]:
df_neighborhoods.to_csv("df_neighborhoods.csv")

<a id="section2"></a>
# 2. Get the coordinates of the neighborhoods

#### Reading data from Geospatial_Data file from http://cocl.us/Geospatial_data

In [21]:
import wget
url = "http://cocl.us/Geospatial_data"
filename = wget.download(url)
print(' Data downloaded!')

100% [................................................................................] 2891 / 2891 Data downloaded!


In [22]:
df_coordinates = pd.read_csv("Geospatial_Coordinates.csv")
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge two dataframes to add coordinates

In [23]:
geo_result = (pd.merge(df_neighborhoods, df_coordinates, left_on="PostalCode", right_on="Postal Code").
            drop('Postal Code', axis=1))

In [24]:
geo_result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


<a id="section3"></a>
# 3. Explorer the neighborhoods in Toronto

#### Select Neighborhoods in Toronto

In [25]:
# Count unique borough
for borough in geo_result['Borough'].unique():
    print(borough,len(geo_result.loc[geo_result['Borough'] == borough]['PostalCode'].unique()))

North York 24
Downtown Toronto 18
Queen's Park 1
Etobicoke 12
Scarborough 17
East York 5
York 5
East Toronto 5
West Toronto 6
Central Toronto 9
Mississauga 1


In [26]:
df_only_toronto = geo_result[geo_result["Borough"].str.contains("Toronto")]
df_only_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [27]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_only_toronto['Borough'].unique()),
        df_only_toronto.shape[0]
    )
)
print("Name of selected boroughs: ", df_only_toronto.Borough.unique())

The dataframe has 4 boroughs and 74 neighborhoods.
Name of selected boroughs:  ['Downtown Toronto' 'East Toronto' 'West Toronto' 'Central Toronto']


#### Explore Neighborhoods in Toronto

In [28]:
import folium #library for making maps

In [29]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.67, -79.37], zoom_start=12)
map_toronto

In [30]:
# Add markers to map
for lat, lng, postal, neighborhood in zip(df_only_toronto['Latitude'], df_only_toronto['Longitude'],
                                          df_only_toronto['PostalCode'], df_only_toronto['Neighborhood']):
    label = '{}, {}'.format(postal,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [61]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Let's get the venues that are in Marble Hill within a radius of 500 meters

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=df_only_toronto['Neighborhood'],
                                   latitudes=df_only_toronto['Latitude'],
                                   longitudes=df_only_toronto['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(3275, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Count the number of venues for each neighborhood

In [35]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,13,13,13,13,13,13
Berczy Park,59,59,59,59,59,59
Brockton,21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
CN Tower,13,13,13,13,13,13
Cabbagetown,44,44,44,44,44,44
Central Bay Street,78,78,78,78,78,78
Chinatown,98,98,98,98,98,98


#### Count of unique categories of from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,CN Tower,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.012821,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010204,0.000000,0.00,0.051020,0.000000,0.051020,0.010204,0.000000,0.000000,0.010204
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.15
1  Airport Terminal  0.15
2   Airport Service  0.15
3          Boutique  0.08
4   Harbor / Marina  0.08


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Seafood Restaurant  0.03
4              Bakery  0.03


----Brockton----
               venue  freq
0     Breakfast Spot  0.10
1               Café  0.10
2        Coffee Shop  0.10
3  Convenience Store  0.05
4         Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1                 Spa  0.06
2       Auto Workshop  0.06
3                Park  0.06
4          Comic Shop  0.06


----CN Tower----
              venue  freq
0    Airp

                 venue  freq
0          Coffee Shop  0.13
1                  Pub  0.13
2          Pizza Place  0.07
3  American Restaurant  0.07
4           Bagel Shop  0.07


----Regent Park----
         venue  freq
0  Coffee Shop  0.16
1         Café  0.08
2          Pub  0.06
3         Park  0.06
4       Bakery  0.06


----Richmond----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Riverdale----
                venue  freq
0    Greek Restaurant  0.20
1         Coffee Shop  0.09
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.07
4           Bookstore  0.05


----Roncesvalles----
                         venue  freq
0               Breakfast Spot  0.13
1                    Gift Shop  0.13
2             Cuban Restaurant  0.07
3           Italian Restaurant  0.07
4  Eastern European Restaurant  0.07


----Rosedale----
            venue  freq
0            Park  0.50
1      Pl

#### A function to sort the venues in descending order

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a new dataframe and display the top 10 venues for each neighborhood

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
2,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Farmers Market,Cheese Shop,Seafood Restaurant,Italian Restaurant,Bakery,Steakhouse
3,Brockton,Café,Coffee Shop,Breakfast Spot,Gym,Climbing Gym,Falafel Restaurant,Convenience Store,Burrito Place,Stadium,Bar
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Garden,Skate Park,Brewery,Comic Shop,Fast Food Restaurant,Spa,Burrito Place,Butcher
5,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
6,Cabbagetown,Coffee Shop,Restaurant,Park,Pizza Place,Bakery,Italian Restaurant,Café,Market,Pub,Breakfast Spot
7,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa,Ice Cream Shop
8,Chinatown,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Bakery,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Farmers Market
9,Christie,Grocery Store,Café,Park,Coffee Shop,Diner,Italian Restaurant,Restaurant,Nightclub,Convenience Store,Baby Store


<a id="section4"></a>
# 4. Cluster the neighborhoods in Toronto

#### Run *k*-means to cluster the neighborhood into 4 clusters

In [42]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [43]:
df_only_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [44]:
# add clustering labels
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    print("clusters added")
except: 
    del neighborhoods_venues_sorted['Cluster Labels']
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    print("old clusters deleted and new clusters added")    

clusters added


In [45]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Adelaide,Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
1,2,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
2,0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Farmers Market,Cheese Shop,Seafood Restaurant,Italian Restaurant,Bakery,Steakhouse
3,0,Brockton,Café,Coffee Shop,Breakfast Spot,Gym,Climbing Gym,Falafel Restaurant,Convenience Store,Burrito Place,Stadium,Bar
4,0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Garden,Skate Park,Brewery,Comic Shop,Fast Food Restaurant,Spa,Burrito Place,Butcher
5,2,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
6,0,Cabbagetown,Coffee Shop,Restaurant,Park,Pizza Place,Bakery,Italian Restaurant,Café,Market,Pub,Breakfast Spot
7,0,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa,Ice Cream Shop
8,0,Chinatown,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Bakery,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Farmers Market
9,0,Christie,Grocery Store,Café,Park,Coffee Shop,Diner,Italian Restaurant,Restaurant,Nightclub,Convenience Store,Baby Store


In [46]:
toronto_merged = df_only_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(50) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Dessert Shop
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Dessert Shop
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Restaurant,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Restaurant,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Bakery,Gastropub,Park,Italian Restaurant,Cosmetics Shop
36,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
37,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Pub,Farmers Market,Cheese Shop,Seafood Restaurant,Italian Restaurant,Bakery,Steakhouse
41,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa,Ice Cream Shop
42,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Diner,Italian Restaurant,Restaurant,Nightclub,Convenience Store,Baby Store
49,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,0,Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant


In [54]:
# create map
map_clusters = folium.Map(location=[43.67, -79.37], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine each cluster and determine the discriminating venue categories that distinguish each cluster

#### Cluster 1

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Dessert Shop
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Dessert Shop
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Restaurant,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Restaurant,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Bakery,Gastropub,Park,Italian Restaurant,Cosmetics Shop
36,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
37,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Pub,Farmers Market,Cheese Shop,Seafood Restaurant,Italian Restaurant,Bakery,Steakhouse
41,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa,Ice Cream Shop
42,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Diner,Italian Restaurant,Restaurant,Nightclub,Convenience Store,Baby Store
49,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,0,Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant


#### Conclusion: Cluster 1 is most probably mixed locality which is lined with restaurants, coffee shops, café's, park and shops

#### Cluster 2

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
150,M4T,Central Toronto,Moore Park,43.689574,-79.38316,1,Playground,Tennis Court,Women's Store,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
151,M4T,Central Toronto,Summerhill East,43.689574,-79.38316,1,Playground,Tennis Court,Women's Store,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Conclusion: Cluster 2 is a neighbourhood with playgrounds and a tenniscourt

#### Cluster 3

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
164,M5V,Downtown Toronto,CN Tower,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
165,M5V,Downtown Toronto,Bathurst Quay,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
166,M5V,Downtown Toronto,Island airport,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
167,M5V,Downtown Toronto,Harbourfront West,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
168,M5V,Downtown Toronto,King and Spadina,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
169,M5V,Downtown Toronto,Railway Lands,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden
170,M5V,Downtown Toronto,South Niagara,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden


#### Conclusion: Cluster 3 is a neighbourhood for transfers (airport, harbor)

#### Cluster 4

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
122,M5P,Central Toronto,Forest Hill North,43.696948,-79.411307,3,Park,Trail,Sushi Restaurant,Jewelry Store,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
123,M5P,Central Toronto,Forest Hill West,43.696948,-79.411307,3,Park,Trail,Sushi Restaurant,Jewelry Store,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
184,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Conclusion: Cluster 4 is a neighbourhood with parks, sushi restaurants, jewelry and some special restaurants. This cluster is similar to cluster 2

In [60]:
toronto_merged.to_csv("toronto_merged.csv")